In [10]:
import numpy as np 
import warnings
from tabulate import tabulate
warnings.filterwarnings("ignore", category = np.VisibleDeprecationWarning) 
allCollections = {}
hashedKeys = {}
bTreedKeys = {}

In [11]:
def printTable(collection, limit = 10):
    # Finds the column names of the collection
    headers = collection.dtype.names
    # Forms a table in a pretty manner
    table = tabulate(collection[: limit], headers, tablefmt = "grid")
    print(table)
    print("Number of rows returned: " + str(len(collection)))

In [12]:
def readFromFile(outputCollection, readFromFile):
    # Read data from a file in which the first line is the name of columns and each column is followed by a |
    collection = np.genfromtxt(readFromFile, dtype = None, delimiter = '|', names = True, autostrip = True)
    # Stores the read collection into a global variable that can be accessed anywhere in this file
    allCollections[outputCollection] = collection
    print(allCollections.keys())
    printTable(allCollections[outputCollection])
    
readFromFile('R', 'sales1')
readFromFile('S', 'sales2')

dict_keys(['R'])
+----------+----------+--------------+-----------+--------+-------+--------------+
|   saleid |   itemid |   customerid |   storeid |   time |   qty | pricerange   |
+==========+==========+==============+===========+========+=======+==============+
|       36 |       14 |            2 |        38 |     49 |    15 | moderate     |
+----------+----------+--------------+-----------+--------+-------+--------------+
|      784 |       90 |          182 |        97 |     46 |    31 | moderate     |
+----------+----------+--------------+-----------+--------+-------+--------------+
|      801 |      117 |            2 |        43 |     81 |    14 | outrageous   |
+----------+----------+--------------+-----------+--------+-------+--------------+
|      905 |       79 |          119 |        81 |     67 |    44 | outrageous   |
+----------+----------+--------------+-----------+--------+-------+--------------+
|      227 |       68 |            2 |        66 |     67 |    42 | su

In [13]:
def findDataType(param):
    try:
        num = int(param)
    except ValueError:
        try:
            num = float(param)
        except ValueError:
            num = str.encode(param)
    return num

In [14]:
def findComparatorMeaning(comparator, collection):
    if comparator in collection.dtype.names:
        return collection[comparator]
    
    elif '+' in comparator:
        comparator = comparator.split('+')
        comparator = [i.strip() for i in comparator]
        if comparator[0] in collection.dtype.names:
            return collection[comparator[0]] + findDataType(comparator[1])
        else:
            return findDataType(comparator[0]) + findDataType(comparator[1])
        
    elif '-' in comparator: 
        comparator = comparator.split('-')
        comparator = [i.strip() for i in comparator]
        if comparator[0] in collection.dtype.names:
            print('yo')
            return collection[comparator[0]] - findDataType(comparator[1])
        else:
            print('yo2')
            return findDataType(comparator[0]) - findDataType(comparator[1])
        
    elif '*' in comparator: 
        comparator = comparator.split('*')
        comparator = [i.strip() for i in comparator]
        if comparator[0] in collection.dtype.names:
            return collection[comparator[0]] * findDataType(comparator[1])
        else:
            return findDataType(comparator[0]) * findDataType(comparator[1])
    elif '/' in comparator:
        comparator = comparator.split('/')
        comparator = [i.strip() for i in comparator]
        if comparator[0] in collection.dtype.names:
            return collection[comparator[0]] / findDataType(comparator[1])
        else:
            return findDataType(comparator[0]) / findDataType(comparator[1])
    else:
        return findDataType(comparator)

In [ ]:
def canEval(expression):
    try:
        eval(expression)
        return True
    except:
        return False

In [29]:
def select(outputCollection, collectionName, conditions, operator):
    allResults = []
    collection = allCollections[collectionName]
    for condition in conditions:     
        # TBD: Add Eval for btree and hash
        if condition[1] == '=' and collectionName + '.' + condition[0] in hashedKeys and canEval(condition[2]):
            print('Using the Hash generated earlier to check for the value')
            compare = eval(condition[2])
            result = hashMatch(collectionName, condition[0], compare)
        elif condition[1] == '=' and collectionName + '.' + condition[2] in hashedKeys and canEval(condition[0]):
            print('Using the Hash generated earlier to check for the value')
            compare = eval(condition[0])
            result = hashMatch(collectionName, condition[2], compare)
        elif condition[1] != '!=' and collectionName + '.' + condition[0] in bTreedKeys and canEval(condition[2]):
            print('Using the Btree generated earlier to check for the value')
            compare = eval(condition[2])
            result = bTreeMatch(collectionName, condition[0], condition[1], compare)
        elif condition[1] != '!=' and collectionName + '.' + condition[2] in bTreedKeys and canEval(condition[0]):
            print('Using the Btree generated earlier to check for the value')
            compare = eval(condition[0])
            result = bTreeMatch(collectionName, condition[2], condition[1], compare)
        else:
            lhs = findComparatorMeaning(condition[0], collection)
            rhs = findComparatorMeaning(condition[2], collection)   
            if condition[1] == '<':
                result = lhs < rhs
            elif condition[1] == '>':
                result = lhs > rhs
            elif condition[1] == '<=':
                result = lhs <= rhs
            elif condition[1] == '>=':
                result = lhs >= rhs
            elif condition[1] == '!=':
                result = lhs != rhs
            elif condition[1] == '=':
                result = lhs == rhs
        allResults.append(result)

    allResults = np.asarray(allResults)
    if operator == 'and':
        finalOutcome = np.extract(np.all(allResults, axis = 0), collection)
    else:
        finalOutcome = np.extract(np.any(allResults, axis = 0), collection)
    if outputCollection == None:
        return finalOutcome
    else:
        allCollections[outputCollection] = finalOutcome
        printTable(allCollections[outputCollection])

In [30]:
select('R1','R', [['time', '>', '50'], ['qty', '<', '30']], 'or')

+----------+----------+--------------+-----------+--------+-------+--------------+
|   saleid |   itemid |   customerid |   storeid |   time |   qty | pricerange   |
+==========+==========+==============+===========+========+=======+==============+
|       36 |       14 |            2 |        38 |     49 |    15 | moderate     |
+----------+----------+--------------+-----------+--------+-------+--------------+
|      801 |      117 |            2 |        43 |     81 |    14 | outrageous   |
+----------+----------+--------------+-----------+--------+-------+--------------+
|      905 |       79 |          119 |        81 |     67 |    44 | outrageous   |
+----------+----------+--------------+-----------+--------+-------+--------------+
|      227 |       68 |            2 |        66 |     67 |    42 | supercheap   |
+----------+----------+--------------+-----------+--------+-------+--------------+
|      951 |      102 |          116 |        45 |     35 |     1 | outrageous   |
+---

In [17]:
whatIGet = {'outputDB': 'T1', 
            'functionName': 'join', 
            'input':'R1',
            'fields': ['S', ['R1.qty', '>', 'S.Q'], ['R1.saleid', '=' ,'S.saleid']],
            'condition': 'and' }

In [18]:
import re
from copy import copy, deepcopy
leftCollection = whatIGet["input"]
print(leftCollection)
rightCollection = whatIGet["fields"][0]
print(rightCollection)
conditions = whatIGet["fields"][1:]
print(conditions)
operator = whatIGet["condition"]
print(operator)

R1
S
[['R1.qty', '>', 'S.Q'], ['R1.saleid', '=', 'S.saleid']]
and


In [19]:
replaceAll = []
for condition in conditions:
    replace = []
    if leftCollection in condition[0].split('.'):
        lhs = condition[0]
        rhs = condition[2]
        posOfLeftCollection = 0
    elif leftCollection in condition[2].split('.'):
        lhs = condition[2]
        rhs = condition[0]
        posOfLeftCollection = 2

    left = re.findall(leftCollection + '.[a-zA-Z_$0-9]+', lhs)
    splitted_left = left[0].split('.')
    replace = [posOfLeftCollection, left[0], leftCollection + '_' + splitted_left[1]]
    right = re.findall(rightCollection + '.[a-zA-Z_$0-9]+', rhs)
    right = right[0].split('.')
    condition[2 - posOfLeftCollection] = re.sub(rightCollection + '.[a-zA-Z_$0-9]+', right[1], rhs)
    replaceAll.append(replace)

In [20]:
print(conditions)
print(replaceAll)

[['R1.qty', '>', 'Q'], ['R1.saleid', '=', 'saleid']]
[[0, 'R1.qty', 'R1_qty'], [0, 'R1.saleid', 'R1_saleid']]


In [21]:
from numpy.lib import recfunctions as rfn
fieldRename = {} 
for name in allCollections[leftCollection].dtype.names:
    fieldRename[name] = leftCollection + '_' + name
leftCollectionRenamed = rfn.rename_fields(allCollections[leftCollection], fieldRename)

fieldRename = {} 
for name in allCollections[rightCollection].dtype.names:
    fieldRename[name] = rightCollection + '_' + name
# allCollections['JoinTemp'] = rfn.rename_fields(allCollections[rightCollection], fieldRename)

In [22]:
leftCollectionRenamed.dtype.names

('R1_saleid',
 'R1_itemid',
 'R1_customerid',
 'R1_storeid',
 'R1_time',
 'R1_qty',
 'R1_pricerange')

In [24]:
joinedCollection = []
firstTime = 1
count = 0
for row in leftCollectionRenamed:
    rowCondition = deepcopy(conditions)
    for i in range(len(rowCondition)):
        pos = replaceAll[i][0]
        rowCondition[i][pos] = rowCondition[i][pos].replace(replaceAll[i][1], str(row[replaceAll[i][2]]))
    columnsThatMatch = select(None, rightCollection, rowCondition, operator)
    if len(columnsThatMatch) > 0:
        columnsThatMatch = rfn.rename_fields(columnsThatMatch, fieldRename)
        joins = rfn.merge_arrays((np.array([row]*len(columnsThatMatch)), columnsThatMatch), flatten = True, usemask = False)
        if firstTime:
            joinedCollection = joins
            firstTime = 0
        else:
            joinedCollection = np.concatenate((joinedCollection, joins), axis = 0)
    count += 1
    print(str(count) + ' : ' + str(len(columnsThatMatch)))

            
        

1 : 0
2 : 0
3 : 1
4 : 1
5 : 0
6 : 0
7 : 0
8 : 0
9 : 0
10 : 0
11 : 0
12 : 0
13 : 1
14 : 0
15 : 1
16 : 1
17 : 1
18 : 1
19 : 1
20 : 1
21 : 1
22 : 1
23 : 0
24 : 0
25 : 1
26 : 0
27 : 0
28 : 1
29 : 1
30 : 0
31 : 1
32 : 0
33 : 1
34 : 0
35 : 0
36 : 0
37 : 0
38 : 0
39 : 0
40 : 0
41 : 0
42 : 0
43 : 1
44 : 0
45 : 0
46 : 1
47 : 1
48 : 0
49 : 1
50 : 0
51 : 1
52 : 1
53 : 1
54 : 1
55 : 1
56 : 1
57 : 1
58 : 1
59 : 0
60 : 0
61 : 0
62 : 1
63 : 1
64 : 0
65 : 1
66 : 1
67 : 0
68 : 0
69 : 1
70 : 0
71 : 1
72 : 0
73 : 0
74 : 1
75 : 0
76 : 1
77 : 0
78 : 0
79 : 1
80 : 0
81 : 0
82 : 0
83 : 0
84 : 0
85 : 1
86 : 0
87 : 0
88 : 1
89 : 0
90 : 0
91 : 0
92 : 1
93 : 0
94 : 0
95 : 0
96 : 0
97 : 0
98 : 0
99 : 0
100 : 1
101 : 1
102 : 0
103 : 0
104 : 1
105 : 0
106 : 0
107 : 0
108 : 1
109 : 0
110 : 1
111 : 1
112 : 0
113 : 1
114 : 0
115 : 0
116 : 0
117 : 1
118 : 0
119 : 0
120 : 0
121 : 1
122 : 1
123 : 0
124 : 0
125 : 0
126 : 1
127 : 1
128 : 0
129 : 1
130 : 0
131 : 0
132 : 0
133 : 1
134 : 0
135 : 0
136 : 1
137 : 1
138 : 0
139 

In [25]:
joinedCollection

array([(905,  79, 119,  81,  67, 44, b'outrageous', 905,  9852,  5746,   2, 14, 40, b'moderate'),
       (227,  68,   2,  66,  67, 42, b'supercheap', 227,  6963,  5746,  37, 73, 27, b'supercheap'),
       (685,  89,  51,  66,  67, 22, b'outrageous', 685, 12333,  8397,  48, 73,  5, b'supercheap'),
       (699,  18,   2,   7,  67, 25, b'outrageous', 699, 18550,  8397,  31, 16, 11, b'supercheap'),
       (472,  95, 174,  62,  67, 43, b'outrageous', 472, 11578, 17050,  89, 73, 42, b'supercheap'),
       (193, 102,  51,  77,  67, 16, b'moderate', 193, 11387, 17551,  65, 16, 10, b'supercheap'),
       (682,  90, 193,  28,  67, 34, b'outrageous', 682,  9629,  6129,  20, 73, 24, b'supercheap'),
       (765, 156,  59,  47,  67,  9, b'expensive', 765,  9264, 10702,  54, 73,  2, b'moderate'),
       (871, 116,  59,  57,  99, 25, b'outrageous', 871,  9796,  5320,  18, 73,  5, b'supercheap'),
       (521, 120, 140,   3,  67, 45, b'expensive', 521, 11667, 16153,  49, 53, 34, b'supercheap'),
       (

In [28]:
eval('5 + 3.0')

8.0